## Sales forecasting using linear models

In this notebook we use simple linear models to predict the weekly sales for the last 8 weeks.

##### Import of the required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import linearmodels as lm
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

%matplotlib inline

##### Definition of helper funtions

In [2]:
def error_table(model_name, y_real, y_pred):
    mae = mean_absolute_error(y_real, y_pred)
    mape = mean_absolute_percentage_error(y_real, y_pred)
    rmse = np.sqrt(mean_squared_error(y_real, y_pred))

    return pd.DataFrame({'Model Name': model_name,
                        'MAE': mae,
                        'MAPE': mape,
                        'RMSE': rmse}, index = [model_name])

##### Data Preparation and Feature Engineering

In [3]:
df = pd.read_csv("../data/sales_stores.csv", parse_dates=['Date'])
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,WeekOfYear,...,StoreType,Assortment,CompetitionDistance,CompetitionOpenSince,Promo2,Promo2Since,PromoInterval,CompetitionTimeDays,CompetitionTimeMonths,PromoTimeWeeks
0,1,5,2015-07-31,5263,555,1,1,no_holiday,1,31,...,c,basic,1270.0,2008-09-01,0,NaN,NaN,2524,82,0
1,2,5,2015-07-31,6064,625,1,1,no_holiday,1,31,...,a,basic,570.0,2007-11-01,1,2010-03-29,"Jan,Apr,Jul,Oct",2829,93,278
2,3,5,2015-07-31,8314,821,1,1,no_holiday,1,31,...,a,basic,14130.0,2006-12-01,1,2011-04-04,"Jan,Apr,Jul,Oct",3164,104,225
3,4,5,2015-07-31,13995,1498,1,1,no_holiday,1,31,...,c,extended,620.0,2009-09-01,0,NaN,NaN,2159,70,0
4,5,5,2015-07-31,4822,559,1,1,no_holiday,1,31,...,a,basic,29910.0,2015-04-01,0,NaN,NaN,121,3,0


During the exploratory data analysis, we realized that some variables had missing values and their effect on the dependent variable (`Sales`) was not easy to interpret. For the sake of simplicity we will leave out these variables out of our modelling (at least for our first attempts).

In [4]:
df.drop(['Promo2', 'Promo2Since', 'PromoInterval', 'CompetitionDistance', 'CompetitionOpenSince', 'CompetitionTimeDays', 'CompetitionTimeMonths'], axis=1, inplace=True)

Because weekyear is cyclical in nature, I will like to code this variable in a way that reflects this cyclical nature. Sine and cosine functions are natural candidates:

In [5]:
# I write a more general funtion in case I need to reuse this functionality later
def encode_cyclical(data, cols, period):
    for col in cols:
        data[col + '_sin'] = np.sin(2 * np.pi * data[col]/period)
        data[col + '_cos'] = np.cos(2 * np.pi * data[col]/period)
    return data

In [6]:
df = encode_cyclical(df, ['WeekOfYear'], df['WeekOfYear'].max())
df.drop('WeekOfYear', axis=1, inplace=True)

One-hot encoding for categorical variables:

In [7]:
df = pd.get_dummies(df, dtype='int')
# No holiday = all zeroes on the StateHoliday columns
df.drop('StateHoliday_no_holiday', axis=1, inplace=True)
df.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'SchoolHoliday', 'PromoTimeWeeks', 'WeekOfYear_sin', 'WeekOfYear_cos',
       'StateHoliday_christmas', 'StateHoliday_easter',
       'StateHoliday_public_holiday', 'Quarter_2013Q1', 'Quarter_2013Q2',
       'Quarter_2013Q3', 'Quarter_2013Q4', 'Quarter_2014Q1', 'Quarter_2014Q2',
       'Quarter_2014Q3', 'Quarter_2014Q4', 'Quarter_2015Q1', 'Quarter_2015Q2',
       'Quarter_2015Q3', 'StoreType_a', 'StoreType_b', 'StoreType_c',
       'StoreType_d', 'Assortment_basic', 'Assortment_extended',
       'Assortment_extra'],
      dtype='object')

In the EDA we noticed that sales are generally higher on Sunday. Because of that we create a feature to capture if a store is open on that day 

In [8]:
df['WeekDay'] = df['Date'].dt.dayofweek
df['SundayOpen'] = df.apply(lambda x: 1 if x['WeekDay']== 6 else 0 , axis = 1)
df.drop('WeekDay', axis=1, inplace=True)

##### Weekly aggregation

We make `Date` the index:

In [9]:
df.set_index('Date', inplace=True)
# Now we have a DataFrame for each date:
#df.loc['2013-01-01']

Let's aggregate the data on a weekly basis:

In [10]:
# The anchored offset W would give sunday weekly frequency
by_week_store = df.groupby([pd.Grouper(freq='W-Mon'), 'Store'])

sum_cols = by_week_store[['Sales', 'Customers', 'Open', 'Promo', 'SchoolHoliday', 'StateHoliday_christmas', 
                          'StateHoliday_easter', 'StateHoliday_public_holiday']].sum()

fix_cols = by_week_store[['StoreType_a', 'StoreType_b', 'StoreType_c', 'StoreType_d', 'Assortment_basic', 
                          'Assortment_extended', 'Assortment_extra', 'WeekOfYear_sin', 'WeekOfYear_cos', 
                          'PromoTimeWeeks', 'SundayOpen']].first()

week_df = pd.concat([sum_cols, fix_cols], axis=1)

week_df.head()

# With code like this we can check that it has worked fine:
#foo = df.loc['2013-01-01':'2013-01-07']
#foo[foo['Store']  == 1]['Sales'].sum()

Sales  Customers  Open  Promo  SchoolHoliday  \
Date       Store                                                 
2013-01-07 1      26516       3285     5      1              5   
           2      22182       2866     5      1              3   
           3      35564       3820     5      1              3   
           4      48928       6985     5      1              3   
           5      20742       2520     5      1              1   

                  StateHoliday_christmas  StateHoliday_easter  \
Date       Store                                                
2013-01-07 1                           0                    0   
           2                           0                    0   
           3                           0                    0   
           4                           0                    0   
           5                           0                    0   

                  StateHoliday_public_holiday  StoreType_a  StoreType_b  \
Date       Store                                                          
2013-01-07 1                                0            0            0   
           2                                0            1            0   
           3                                0            1            0   
           4                                0            0            0   
           5                                0            1            0   

                  StoreType_c  StoreType_d  Assortment_basic  \
Date       Store                                               
2013-01-07 1                1            0                 1   
           2                0            0                 1   
           3                0            0                 1   
           4                1            0                 0   
           5                0            0                 1   

                  Assortment_extended  Assortment_extra  WeekOfYear_sin  \
Date       Store                                                          
2013-01-07 1                        0                 0        0.239316   
           2                        0                 0        0.239316   
           3                        0                 0        0.239316   
           4                        1                 0        0.239316   
           5                        0                 0        0.239316   

                  WeekOfYear_cos  PromoTimeWeeks  SundayOpen  
Date       Store                                              
2013-01-07 1            0.970942               0           0  
           2            0.970942             145           0  
           3            0.970942              92           0  
           4            0.970942               0           0  
           5            0.970942               0           0

`Date`now shows us the Monday of the corresponding week, while the remaining variables each show the weekly total of the previous week. But I would like to have the previous Monday instead as the date.

In [11]:
# Return temporarily to a sequential index
week_df = week_df.reset_index()
week_df['Date'] = week_df['Date'] - pd.Timedelta(days=7)
# Return to multiindex
week_df.sort_values(by=['Store', 'Date'], ascending=[True, False], inplace=True)
week_df.set_index(['Store', 'Date'], inplace=True)
week_df.head()

Sales  Customers  Open  Promo  SchoolHoliday  \
Store Date                                                       
1     2015-07-27  20076       2184     4      4              4   
      2015-07-20  24963       3006     6      1              1   
      2015-07-13  27889       3168     6      4              0   
      2015-07-06  23736       2893     6      1              0   
      2015-06-29  30337       3341     6      4              0   

                  StateHoliday_christmas  StateHoliday_easter  \
Store Date                                                      
1     2015-07-27                       0                    0   
      2015-07-20                       0                    0   
      2015-07-13                       0                    0   
      2015-07-06                       0                    0   
      2015-06-29                       0                    0   

                  StateHoliday_public_holiday  StoreType_a  StoreType_b  \
Store Date                                                                
1     2015-07-27                            0            0            0   
      2015-07-20                            0            0            0   
      2015-07-13                            0            0            0   
      2015-07-06                            0            0            0   
      2015-06-29                            0            0            0   

                  StoreType_c  StoreType_d  Assortment_basic  \
Store Date                                                     
1     2015-07-27            1            0                 1   
      2015-07-20            1            0                 1   
      2015-07-13            1            0                 1   
      2015-07-06            1            0                 1   
      2015-06-29            1            0                 1   

                  Assortment_extended  Assortment_extra  WeekOfYear_sin  \
Store Date                                                                
1     2015-07-27                    0                 0       -0.568065   
      2015-07-20                    0                 0       -0.568065   
      2015-07-13                    0                 0       -0.464723   
      2015-07-06                    0                 0       -0.354605   
      2015-06-29                    0                 0       -0.239316   

                  WeekOfYear_cos  PromoTimeWeeks  SundayOpen  
Store Date                                                    
1     2015-07-27       -0.822984               0           0  
      2015-07-20       -0.822984               0           0  
      2015-07-13       -0.885456               0           0  
      2015-07-06       -0.935016               0           0  
      2015-06-29       -0.970942               0           0

In [12]:
# Save this data for later use
week_df.to_csv("../data/week_sales.csv", index=False)

##### Split dataframe into training and test

In [13]:
# 8 weeks before the last date
test_date = week_df.index[0][1]- datetime.timedelta(weeks=7)

predictors = [x for x in week_df.columns if x not in ['Sales', 'Customers']]

X_test =week_df[week_df.index.get_level_values(1) >= test_date][predictors]
X_train =week_df[week_df.index.get_level_values(1) < test_date][predictors]

y_test =week_df['Sales'][week_df.index.get_level_values(1) >= test_date]
y_train =week_df['Sales'][week_df.index.get_level_values(1) < test_date]

In [14]:
# observations per store
# X_train.groupby('Store').size().unique()

We create weekly `AvgSales` and `AvgCustomers`  and `SalesPerCustomer` as features that characterize individual stores taking into account only the data before the last 8 weeks. These features could be continually updated in a productive environment:

In [15]:
avgs = week_df[week_df.index.get_level_values(1) < test_date].groupby(level=0)[['Sales', 'Customers']].mean()
avgs.rename({'Sales': 'AvgSales', 'Customers': 'AvgCustomers'}, axis=1, inplace=True)
# This is the same as total sales / total customers. We will only use train data 
avgs['SalesPerCustomer'] = avgs.apply(lambda x: x['AvgSales'] / x['AvgCustomers'], axis=1)

X_test = X_test.join(avgs)
X_train = X_train.join(avgs)

##### Averages Model

Our baseline model will be a model which predicts the average sales of each store.

In [16]:
y_avg = X_test['AvgSales']
error_table('Averages Model', y_test, y_avg)

,Model Name,MAE,MAPE,RMSE
Averages Model,Averages Model,4218.377087,0.10822,5810.060655


##### Scaling

For more complex models, we must log-scale `Sales`, `AvgSales`  and `AvgCustomers` because they have a large range of values and their distributions are skewed towards long values.

In [17]:
y_test = np.log1p(y_test)
y_train = np.log1p(y_train)
X_test[['AvgSales', 'AvgCustomers']] = np.log1p(X_test[['AvgSales', 'AvgCustomers']])
X_train[['AvgSales', 'AvgCustomers']] = np.log1p(X_train[['AvgSales', 'AvgCustomers']])

In [18]:
mms = MinMaxScaler()
cols_to_scale = ['Open', 'Promo', 'SchoolHoliday', 'StateHoliday_christmas', 'StateHoliday_easter', 
                 'StateHoliday_public_holiday']
X_train[cols_to_scale] = mms.fit_transform(X_train[cols_to_scale])
X_test[cols_to_scale] = mms.fit_transform(X_test[cols_to_scale])

In [19]:
X_train.head()

Open  Promo  SchoolHoliday  StateHoliday_christmas  \
Store Date                                                                 
1     2015-06-01  0.666667    0.6            0.0                     0.0   
      2015-05-25  0.833333    0.2            0.0                     0.0   
      2015-05-18  0.666667    0.8            0.0                     0.0   
      2015-05-11  0.666667    0.2            0.0                     0.0   
      2015-05-04  0.833333    0.8            0.0                     0.0   

                  StateHoliday_easter  StateHoliday_public_holiday  \
Store Date                                                           
1     2015-06-01                  0.0                          0.0   
      2015-05-25                  0.0                          0.0   
      2015-05-18                  0.0                          0.0   
      2015-05-11                  0.0                          0.0   
      2015-05-04                  0.0                          0.0   

                  StoreType_a  StoreType_b  StoreType_c  StoreType_d  \
Store Date                                                             
1     2015-06-01            0            0            1            0   
      2015-05-25            0            0            1            0   
      2015-05-18            0            0            1            0   
      2015-05-11            0            0            1            0   
      2015-05-04            0            0            1            0   

                  Assortment_basic  Assortment_extended  Assortment_extra  \
Store Date                                                                  
1     2015-06-01                 1                    0                 0   
      2015-05-25                 1                    0                 0   
      2015-05-18                 1                    0                 0   
      2015-05-11                 1                    0                 0   
      2015-05-04                 1                    0                 0   

                  WeekOfYear_sin  WeekOfYear_cos  PromoTimeWeeks  SundayOpen  \
Store Date                                                                     
1     2015-06-01        0.239316       -0.970942               0           0   
      2015-05-25        0.354605       -0.935016               0           0   
      2015-05-18        0.568065       -0.822984               0           0   
      2015-05-11        0.568065       -0.822984               0           0   
      2015-05-04        0.663123       -0.748511               0           0   

                   AvgSales  AvgCustomers  SalesPerCustomer  
Store Date                                                   
1     2015-06-01  10.228754       8.09786          8.424653  
      2015-05-25  10.228754       8.09786          8.424653  
      2015-05-18  10.228754       8.09786          8.424653  
      2015-05-11  10.228754       8.09786          8.424653  
      2015-05-04  10.228754       8.09786          8.424653

In [20]:
# Save this data for later use
X_test.to_csv("../data/x_test.csv")
X_train.to_csv("../data/x_train.csv")
y_test.to_csv("../data/y_test.csv")
y_train.to_csv("../data/y_train.csv")

##### Pooled Regression Model with OLS

This is a well known kind of regression in the econometrics literature:

In [21]:
# Add a column with ones
X_pr_train = sm.add_constant(X_train)
X_pr_test = sm.add_constant(X_test)

# avoid dummy variable trap
X_pr_train.drop(['StoreType_a', 'Assortment_basic'], axis=1, inplace=True)
X_pr_test.drop(['StoreType_a', 'Assortment_basic'], axis=1, inplace=True)

In [22]:
# estimate the model
pool_reg = lm.panel.PooledOLS(y_train, X_pr_train).fit()
print(pool_reg)
# estimate the model with another package
#pool_reg = sm.OLS(y_train, X_pr_train).fit()
#print(pool_reg.summary())

                          PooledOLS Estimation Summary                          
Dep. Variable:                  Sales   R-squared:                        0.8715
Estimator:                  PooledOLS   R-squared (Between):              0.9982
No. Observations:              136726   R-squared (Within):               0.2990
Date:                Fri, Jan 03 2025   R-squared (Overall):              0.8715
Time:                        13:28:39   Log-likelihood                   8.8e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                    5.15e+04
Entities:                        1115   P-value                           0.0000
Avg Obs:                       122.62   Distribution:               F(18,136707)
Min Obs:                       97.000                                           
Max Obs:                       127.00   F-statistic (robust):           5.15e+04
                            

In [23]:
y_pr = pool_reg.predict(X_pr_test)
error_table( 'Pooled Regression with OLS', np.expm1(y_test), np.expm1(y_pr))

,Model Name,MAE,MAPE,RMSE
Pooled Regression with OLS,Pooled Regression with OLS,3177.766337,0.079115,4429.824986


##### Simple Linear Regression for each Store

What if we train an independent linear regression for eahc store?

In [24]:
# revert to single index on date
X_train_1 = X_train.reset_index(level=[0])
X_test_1 = X_test.reset_index(level=[0])
y_train_1 = y_train.reset_index(level=[0])
y_test_1 = y_test.reset_index(level=[0])

# drop columns which are fixed for a given store
not_predictors = ['StoreType_a', 'StoreType_b', 'StoreType_c', 'StoreType_d', 'Assortment_basic', 
                  'Assortment_extended', 'Assortment_extra']
X_train_1.drop(not_predictors, axis=1, inplace=True)
X_test_1.drop(not_predictors, axis=1, inplace=True)

y_lr_test_list = []
y_lr_pred_list = []
# loop over stores
for s in np.sort(X_train_1['Store'].unique()):
    # train and test independent variables for the store
    X_test_s = X_test_1[X_test_1['Store'] == s].drop(['Store'], axis=1)
    X_train_s = X_train_1[X_train_1['Store'] == s].drop(['Store'], axis=1)
    # dependent variable for the store
    y_test_s = y_test_1[y_test_1['Store'] == s]['Sales']
    y_train_s = y_train_1[y_train_1['Store'] == s]['Sales']
    # Is good to have also the store as part of the index
    y_test_s.index = pd.MultiIndex.from_tuples([(s, date) for date in X_test_s.index], names=('Store', 'Date'))
    # fit and predict
    lr = LinearRegression().fit(X_train_s, y_train_s)
    y_pred_s = pd.Series(lr.predict(X_test_s), y_test_s.index)
    # append to lists
    y_lr_test_list.append(y_test_s)
    y_lr_pred_list.append(y_pred_s)

# concatenate series
y_lr_test = pd.concat(y_lr_test_list)
y_lr_pred = pd.concat(y_lr_pred_list)

error_table( 'Linear Regression per Store', np.expm1(y_lr_test), np.expm1(y_lr_pred))

,Model Name,MAE,MAPE,RMSE
Linear Regression per Store,Linear Regression per Store,3051.338616,0.077325,4424.518507
